## Imports

In [1]:
import tensorflow as tf
import json
import models
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
EXAMINE = 0

## Load Training Data

In [3]:
torch.manual_seed(22)

blog_posts_data_dir = "data/blogs/json-data/"
train_file_name = "train.json"
test_file_name = "test.json"

training_set = datasets.BlogPostDataset(blog_posts_data_dir, train_file_name)

# Map each word to a unique int value
word_to_int = {}
for instance in training_set:
    for word in instance["post"]:
        if word not in word_to_int:
            word_to_int[word] = len(word_to_int)

def prepare_sequence(seq, word_to_int):
    ints = [word_to_int[w] for w in seq]
    return torch.tensor(ints, dtype = torch.long)


so wuts up? today i had the parade. suked. but it wasnt that bad. im done with band for the year. we had a battle today. we kicked ass.  they had nothing.  then jims party.  then my snotty little cousins bday party. i dun like her. sublime. out.


## Find Key Metrics

In [4]:
samples_count = len(training_set)

categories_count = len(training_set[0]["age"])

samples_per_class = {0 : 0, 1 : 0, 2 : 0}
for instance in training_set:
    for i, a in enumerate(instance["age"]):
        if a == 1:
            samples_per_class[i] += 1
            break

median_words_per_sample = np.median([len(instance["post"]) for instance in training_set])
 

In [5]:
print("Number of Samples:", samples_count)
print("Number of Categories:", categories_count)
print("Samples per Class:", samples_per_class)
print("Median Words per Sample:", median_words_per_sample)
print("Samples to Words Per Sample Ratio:", samples_count / median_words_per_sample)

Number of Samples: 526812
Number of Categories: 3
Samples per Class: {0: 177940, 1: 250672, 2: 98200}
Median Words per Sample: 121.0
Samples to Words Per Sample Ratio: 4353.818181818182


In [6]:
plt.hist(list(length_distribution.keys()))
plt.xlabel("Length of a Sample")
plt.ylabel("Number of samples")
plt.show()

NameError: name 'length_distribution' is not defined

## Train the Model

[Using Pretrained Word Embeddings in PyTorch](https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76)

[An Introduction to Different Types of Convolutions](https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d)

In [7]:

# Train the model
EMBEDDING_DIM = 50
HIDDEN_DIM = 15
NUM_AGE_GROUPS = 3
model = models.BasicLSTMAgeClassifier(EMBEDDING_DIM, word_to_int.keys(), HIDDEN_DIM, NUM_AGE_GROUPS)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

# WORDS DON'T HAVE AGES, SENTENCES HAVE AGES. I figured this out when I was high. Just thought
# where did the "4" in the prediction tensor's shape come from?
# I need to treat myself like a student and write out my thought process as I'm trying to solve
# a problem. I need to ask myself questions.

# See what the scores are before training
with torch.no_grad():
    inputs = prepare_sequence(training_set[EXAMINE]["post"], word_to_int)
    group_scores = model(inputs)
    print(group_scores)

for epoch in range(300):
    for instance in training_set[ : 4000]:
        
        # Zero-out the gradients
        model.zero_grad()

        # Zero-out hidden state from previous instance
        model.hidden = model.init_hidden()

        sentence_in = prepare_sequence(instance["post"], word_to_int)
        group = torch.tensor(instance["age"], dtype = torch.long).unsqueeze(0)
        print("Group Shape:", group.shape)
        print("Group:", group)

        group_scores = model(sentence_in)
        print("Group Scores Shape:", group_scores.shape)
        print("Group Score Training:", group_scores)
        
        loss = loss_function(group_scores, group)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_set[EXAMINE]["post"], word_to_int)
    group_scores = model(inputs)
    print(group_scores)

# TODO: Save the model

128456 out of 2772417 found in GloVe embeddings


TypeError: 'int' object is not iterable